In [30]:
import pandas as pd
from sklearn import svm
from sklearn import tree
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import decomposition
from sklearn.naive_bayes import GaussianNB
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import numpy as np
import matplotlib.pyplot as plt

In [31]:
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df_result = df_test[['PassengerId']].copy()
df_result['Survived'] = pd.Series(np.random.randn(len(df_result)), index=df_result.index)
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
PassengerId    891 non-null int64
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Ticket         891 non-null object
Fare           891 non-null float64
Cabin          204 non-null object
Embarked       889 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 83.6+ KB


In [15]:
"""fig = plt.figure()
ax = fig.add_subplot(1,1,1)
ax.set_xlabel('Survived?')
ax.set_ylabel('No of people')
ax.set_title("Total survived vs not survived")
pd.value_counts(df_train['Survived']).plot.bar()
#df_train['Survived'].plot(kind='bar')
plt.show()"""

'fig = plt.figure()\nax = fig.add_subplot(1,1,1)\nax.set_xlabel(\'Survived?\')\nax.set_ylabel(\'No of people\')\nax.set_title("Total survived vs not survived")\npd.value_counts(df_train[\'Survived\']).plot.bar()\n#df_train[\'Survived\'].plot(kind=\'bar\')\nplt.show()'

In [32]:
def normalizeDf(df):
    temp = df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    temp_scaled = min_max_scaler.fit_transform(temp)
    df = pd.DataFrame(temp_scaled)
    df.columns = ['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
    return df

In [33]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)
fare_mean = df_train['Fare'].mean()
fare_median = df_train['Fare'].median()

for i in range(0,len(df_train['Fare'])):
    val = df_train.iloc[i]['Fare']
    if(val<=fare_median):
        df_train.at[i,'Fare']  = 1
    elif(val>fare_median and val<=fare_mean):
        df_train.at[i,'Fare'] = 2
    else:
        df_train.at[i,'Fare'] = 3

for i in range(0, len(df_test['Fare'])):
    val = df_test.iloc[i]['Fare']
    if(val<=fare_median):
        df_test.at[i,'Fare'] = 1
    elif(val>fare_median and val<=fare_mean):
        df_test.at[i,'Fare'] = 2
    else:
        df_test.at[i,'Fare'] = 3
        
        
for i in range(0,len(df_train['Age'])):
    val = df_train.iloc[i]['Age']
    if(val<=15):
        df_train.at[i,'Age']  = 1
    elif(val>15 and val<=50):
        df_train.at[i,'Age'] = 2
    else:
        df_train.at[i,'Age'] = 3

for i in range(0, len(df_test['Age'])):
    val = df_test.iloc[i]['Age']
    if(val<=15):
        df_test.at[i,'Age'] = 1
    elif(val>15 and val<=50):
        df_test.at[i,'Age'] = 2
    else:
        df_test.at[i,'Age'] = 3

df_train['Sex'].replace(['female','male'],[1,2],inplace=True)
df_test['Sex'].replace(['female','male'],[1,2],inplace=True)
df_train['Embarked'].replace(['Q','S','C'],[1,2,3],inplace=True)
df_test['Embarked'].replace(['Q','S','C'],[1,2,3],inplace=True)

#df_train_mod = normalizeDf(df_train[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']])
#df_test_mod = normalizeDf(df_test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']])
#df_test['Pclass'] = (eg_array - eg_array.mean(axis=0)) / eg_array.std(axis=0)

X = df_train[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
y = df_train['Survived']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.33,random_state=42)

#X_train = df_train[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
#Y_train = df_train['Survived']

X_toTest = df_test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]

In [34]:
X_train.head(6)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
6,1,2,3.0,0,0,3.0,2
718,3,2,1.0,0,0,2.0,1
685,2,2,2.0,1,2,3.0,3
73,3,2,2.0,1,0,1.0,3
882,3,1,2.0,0,0,1.0,2
328,3,1,2.0,1,1,2.0,2


In [19]:
pca = decomposition.PCA()
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)
X_toTest = pca.transform(X_toTest)

In [20]:
tuned_parameters = [{'kernel': ['linear'], 'C': [0.0001,0.001,0.01,0.1, 1, 10]}]
clf = GridSearchCV(svm.SVC(), tuned_parameters, cv=5)

In [35]:
#clf = svm.SVC(kernel='linear',C=0.1)
clf = AdaBoostClassifier(RandomForestClassifier(max_depth=1),
                         algorithm="SAMME",
                         n_estimators=200)
#clf = RandomForestClassifier()
clf.fit(X_train, y_train)
#print(clf.best_params_)

AdaBoostClassifier(algorithm='SAMME',
          base_estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=1, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
          learning_rate=1.0, n_estimators=200, random_state=None)

In [36]:
y_pred = clf.predict(X_toTest)
#y_pred = clf.predict(X_test)
#acc = accuracy_score(y_test,y_pred)
#acc

In [37]:
df_result['Survived'] = y_pred
df_result.to_csv('submission.csv')